In [1]:
from langchain_groq import ChatGroq

In [3]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("PRESIDENT OF THE USA?")
print(response.content)

The current President of the United States is Joe Biden. He was inaugurated as the 46th President of the United States on January 20, 2021.


In [16]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/cdn-security-engineer-waf/job/R-48004")
page_data = loader.load().pop().page_content
print(page_data)






















CDN Security Engineer (WAF)










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



In [19]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 

res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)
print(res.content)

```json
{
  "role": "CDN Security Engineer (WAF)",
  "experience": "At least five (5) or more years of relevant experience, Two (2) or more years of experience providing similar operations and sustainment support with Akamai or other CDN providers",
  "skills": "CDN concepts, Akamai's suite of tools and security offerings, API security requirements, web attack vectors, threat indicators, cloud infrastructure, security",
  "description": "Designing, implementing, and sustaining Nike's cloud infrastructure and security posture, with a focus on Content Delivery Network (CDN) security, collaborating with development and engineering teams to drive standards, innovation, and growth"
}
```


In [20]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'CDN Security Engineer (WAF)',
 'experience': 'At least five (5) or more years of relevant experience, Two (2) or more years of experience providing similar operations and sustainment support with Akamai or other CDN providers',
 'skills': "CDN concepts, Akamai's suite of tools and security offerings, API security requirements, web attack vectors, threat indicators, cloud infrastructure, security",
 'description': "Designing, implementing, and sustaining Nike's cloud infrastructure and security posture, with a focus on Content Delivery Network (CDN) security, collaborating with development and engineering teams to drive standards, innovation, and growth"}

In [24]:
import pandas as pd

df = pd.read_csv("portfolio.csv")
df

,Techstack,Links
0,"Flutter, Dart",https://github.com/RithishDE/BODY_MASS_INDEX_APP
1,"Docker,Kuberneties,Cli",https://github.com/RithishDE/DOCKER_KUBERNETIES
2,"Design Patterns ,oops",https://github.com/RithishDE/DESIGN_PATTERN-_A...
3,"Airflow,Python",https://github.com/RithishDE/Airflow_assignment
4,"Python, spark,pyspark,cli",https://github.com/RithishDE/PYSPARK_ASSIGNMENT
5,"Snowflake, Python",https://github.com/RithishDE/Snow_Flake_Assign...
6,AWS,https://github.com/RithishDE/AWS_ASSIGNMENT
7,Scala,https://github.com/RithishDE/Scala_Assignment
8,MongoDB,https://github.com/RithishDE/MONGO_DB_ASSIGNMENT
9,SQL,https://github.com/RithishDE/SQL_ASSIGNMENT


In [27]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [29]:
links = collection.query(query_texts=json_res['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://github.com/RithishDE/Quonny'},
  {'links': 'https://github.com/RithishDE/SPNDR'}]]

In [30]:
json_res

{'role': 'CDN Security Engineer (WAF)',
 'experience': 'At least five (5) or more years of relevant experience, Two (2) or more years of experience providing similar operations and sustainment support with Akamai or other CDN providers',
 'skills': "CDN concepts, Akamai's suite of tools and security offerings, API security requirements, web attack vectors, threat indicators, cloud infrastructure, security",
 'description': "Designing, implementing, and sustaining Nike's cloud infrastructure and security posture, with a focus on Content Delivery Network (CDN) security, collaborating with development and engineering teams to drive standards, innovation, and growth"}

In [32]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Deva Rithish, a Software Engineer at Sigmoid Analytics with 2 years of Experience.I have projects covering AI,ML, Data engineering. 
        In my experience, I have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of myself
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Rithish's portfolio: {link_list}
        Remember you are Rithish, SDE 1 at Sigmoid Analytics. also at the end say Please find the attached cv for your reference 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(json_res), "link_list": links})
print(res.content)

Subject: Application for CDN Security Engineer (WAF) Role at Nike

Dear Hiring Manager,

I am excited to apply for the CDN Security Engineer (WAF) position at Nike, where I can leverage my skills and experience to design, implement, and sustain a secure cloud infrastructure and security posture. Although I have 2 years of experience as a Software Engineer at Sigmoid Analytics, I am confident that my expertise in AI, ML, and data engineering can be adapted to meet the requirements of this role.

In my current role, I have empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. I believe that my experience in driving standards, innovation, and growth can be valuable in collaborating with development and engineering teams at Nike.

I have a strong foundation in cloud infrastructure and security, which I believe can be applied to CDN concepts and Akamai's suite of tools and security offerings. M